In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import sys
sys.path.append('/Users/mregnier/Desktop/PhD Regnier/mypackages')


import frequency_acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from qubic import SpectroImLib as sp
from importlib import reload
import gc
import copy
from pyoperators import *
import mypcg


# PyOperators packages
from pyoperators import (
    BlockColumnOperator, BlockDiagonalOperator, BlockRowOperator,
    CompositionOperator, DiagonalOperator, I, IdentityOperator,
    MPIDistributionIdentityOperator, MPI, proxy_group, ReshapeOperator,
    rule_manager, pcg, Operator)

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")
# %matplotlib inline

## Now QUBIC and Planck acquisitions used for reconstruction

def integration(nu_min, nu_max, Nintegr, sky_config, d, fwhm_subbands = None):
    print(f'Integration from {nu_min:.2f} to {nu_max:.2f} GHz with {Nintegr} steps')
    obj = Acq.QubicIntegrated(d, Nsub=Nintegr, Nrec=Nintegr)
    if Nintegr == 1:
        allnus = np.array([np.mean([nu_min, nu_max])])
    else:
        allnus = np.linspace(nu_min, nu_max, Nintegr)
    m = obj.get_PySM_maps(sky_config, nus=allnus)
    
    if fwhm_subbands is not None:
        for i in range(Nintegr):
            C = HealpixConvolutionGaussianOperator(fwhm=fwhm_subbands[i])
            m[i] = C(m[i])
    
    return np.array([np.mean(m, axis=0)])


# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


reload(Acq)
nside = 256
relative_bandwidth = 0.25
band = 220
band_planck = 217
seed = 42
noiseless = False
effective_duration = 3e0
rwplanck = 1.
tol = 1e-40
noise_planck_level = 0
noise_qubic_level = 0

def get_dictionary(nsub, nside, pointing, band):
    dictfilename = 'dicts/pipeline_demo.dict'
    
    # Read dictionary chosen
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictfilename)
    d['nf_recon'] = nsub
    d['nf_sub'] = nsub
    d['nside'] = nside
    d['RA_center'] = 0
    d['DEC_center'] = -57
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
    d['effective_duration'] = 3
    d['npointings'] = pointing
    d['filter_nu'] = int(band*1e9)
    d['photon_noise'] = True
    d['noiseless'] = False
    d['config'] = 'FI'
    d['MultiBand'] = True
    
    return d, center

In [ ]:
nsub = 1
nside = 256
pointing = 1000
band = 150
d150, center = get_dictionary(nsub, nside, pointing, 150)
d220, _ = get_dictionary(nsub, nside, pointing, 220)

In [ ]:
reload(Acq)

fact_sub = 1
Nrec = 1
Nsub = fact_sub * Nrec

qubic150 = Acq.QubicIntegrated(d150, Nsub=Nsub, Nrec=Nrec)
qubic220 = Acq.QubicIntegrated(d220, Nsub=Nsub, Nrec=Nrec)

In [ ]:
qu = Acq.QubicTwoBands(qubic150, qubic220)
planck143 = Acq.PlanckAcquisition(143, qu.scene)
planck217 = Acq.PlanckAcquisition(217, qu.scene)

qupl = Acq.QubicPlanckMultiBandAcquisition(qu, [planck143, planck217])
cov = qubic150.get_coverage()
seenpix1 = cov/cov.max() > 0.
seenpix2 = cov/cov.max() > 0.
#mask = np.array(seenpix)
#Hw = qu.get_operator(convolution=False)
#print(Hw.shapein, Hw.shapeout)

In [ ]:
mask = np.array(seenpix2, dtype=float)
mask += 1
mask[seenpix1] *= 1e20

In [ ]:
#invN.operands[2].operands[1]

In [ ]:
invN = qupl.get_invntt_operator()
#invN.operands[0].operands[1] /= 1e0
#invN.operands[2].operands[1] /= 1e0

In [ ]:
Hw_pl = qupl.get_operator()

In [ ]:
cmb = qu.qubic150.get_PySM_maps({'cmb':42}, nus=qu.nueff)
cmb_qu = cmb.copy()
#cmb_qu[:, ~seenpix2] = 0
cmb_pl = cmb.copy()
#cmb_pl[:, seenpix1] = 0

In [ ]:
Hpl = ReshapeOperator((786432,3), 2359296)
tod_pl = Hpl(cmb_pl[0])
n = qupl.get_noise()
n_pl = n[991008:991008 + 2359296]
n_pl = Hpl.T(n_pl)
n_pl[seenpix1] = 0
n_pl = Hpl(n_pl)

In [ ]:
tod = np.array([])
tod = np.append(tod, tod_qubic[:991008])# + n[:991008])
for i in range(Nrec):
    tod = np.append(tod, tod_pl+n_pl)

tod = np.append(tod, tod_qubic[991008 + 2359296:2*991008 + 2359296])# + n[991008 + Nrec * 3*12*nside**2:2*991008 + (Nrec) * 3*12*nside**2])

for i in range(Nrec):
    tod = np.append(tod, tod_pl+n_pl)

In [ ]:


#tod = Hw_pl(cmb) + n


A = Hw_pl.T * invN * Hw_pl
b = Hw_pl.T * invN * tod

In [ ]:
plt.plot(n)
plt.plot(tod, alpha=0.3)

In [ ]:
solution = pcg(A, b, x0=None, maxiter=50, tol=1e-40, disp=True)

In [ ]:
pixok = cov/cov.max() > 0.1
res = 25
r = 1
istk = 1

plt.figure(figsize=(15, 5))

hp.gnomview(cmb[0, :, istk], cmap='jet', rot=center, reso=res, sub=(1, 3, 1), min=-8, max=8)
#solution['x'][0, ~pixok, istk] = hp.UNSEEN
hp.gnomview(solution['x'][0, :, istk], cmap='jet', rot=center, reso=res, sub=(1, 3, 2), min=-8, max=8)

residual = cmb[0, :, istk]-solution['x'][0, :, istk]
#residual[~pixok] = hp.UNSEEN
hp.gnomview(residual, cmap='jet', rot=center, reso=res, sub=(1, 3, 3), min=-r, max=r)

plt.show()

plt.figure(figsize=(15, 5))

hp.gnomview(cmb[1, :, istk], cmap='jet', rot=center, reso=res, sub=(1, 3, 1), min=-8, max=8)
#solution['x'][1, ~pixok, istk] = hp.UNSEEN
hp.gnomview(solution['x'][1, :, istk], cmap='jet', rot=center, reso=res, sub=(1, 3, 2), min=-8, max=8)

residual = cmb[1, :, istk]-solution['x'][1, :, istk]
#residual[~pixok] = hp.UNSEEN
hp.gnomview(residual, cmap='jet', rot=center, reso=res, sub=(1, 3, 3), min=-r, max=r)

plt.show()